# PROBLEM C5

Build and train a neural network model using the Daily Min Temperature.csv dataset.
Use MAE as the metrics of your neural network model.
We provided code for normalizing the data. Please do not change the code.
Do not use lambda layers in your model.

The dataset used in this problem is downloaded from https://github.com/jbrownlee/Datasets

Desired MAE < 0.19 on the normalized dataset.

Prepare Imports

In [ ]:
!pip install numpy==1.24.3

In [ ]:
!pip install pandas==2.0.3

In [ ]:
!pip install Pillow==10.0.0

In [ ]:
!pip install scipy==1.10.1

In [ ]:
!pip install tensorflow==2.13.0

In [ ]:
!pip install tensorflow-datasets==4.9.2

Imports

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import urllib

Get The Data

In [ ]:
data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
urllib.request.urlretrieve(data_url, 'daily-min-temperatures.csv')

In [ ]:
time_step = []
temps = []

In [ ]:
with open('daily-min-temperatures.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        step = 0
        for row in reader:
            temps.append(float(row[1]))
            time_step.append(row[0])
            step=step + 1

In [ ]:
series=np.array(temps)

Normalizing The Series

In [ ]:
min=np.min(series)
max=np.max(series)
series -= min
series /= max
time=np.array(time_step)

Split The Dataset

In [ ]:
split_time=2500

time_train= time[:split_time]
x_train= series[:split_time]
time_valid=  time[split_time:]
x_valid=  series[split_time:]

Create Windowed Dataset

In [ ]:
# Specification for Windowed Dataset and Model
window_size=64
batch_size=256
shuffle_buffer_size=1000

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
train_set=windowed_dataset(
        x_train, window_size, batch_size, shuffle_buffer_size)

In [ ]:
train_set

In [ ]:
x_train.shape

Model Creation and Callback

In [ ]:
 model=tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, input_shape=[None,1]),
        tf.keras.layers.Dense(32),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16),
        # YOUR CODE HERE.
        tf.keras.layers.Dense(1)
    ])

In [ ]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('mae') < 0.08:
            self.model.stop_training = True

In [ ]:
callback = MyCallback

In [ ]:
model.compile(loss='mae', optimizer='sgd',metrics=['mae'])
    model.fit(train_set, epochs=2000, callbacks=[callback])